# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [24]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
import re

In [25]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\berna\C dados projeto1\cdados


Carregando a base de dados com as mensagens dos seus arquivos:

In [26]:
train = pd.read_csv('dados_treino_TRIO_mariaear.csv')
train.head(5)

,Comment,Topic
0,"Hey, have you heard about floating point error...",Physics
1,Are you numb?,Physics
2,thank you!!,Chemistry
3,"just what I was hoping for, thank you!",Chemistry
4,I thought it said dck and immediately thought ...,Chemistry


In [27]:
test = pd.read_csv('dados_teste_TRIO_mariaear.csv')
test.head(5)

,Comment,Topic
0,Well we learned IUPAC it was just never tested...,Chemistry
1,ok what happen when i lick licky or when crazy...,Chemistry
2,"Yeah, but 300 trillion is the total global car...",Physics
3,If only the particles outside the event horizo...,Physics
4,Here's a fun video describing more properties ...,Physics


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

CAsos de seleção quimica e fisica

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [28]:
#limpar tabela
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    pattern2 = re.compile("\n")
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(pattern2, ' ', text_subbed)
    return text_subbed

def cleaning(text):
    # if "\n" in text:
    #     text=text.replace("\n"," ")
    pontuacao='`^´~!#$:;.,[]{}’“”<>?'
    for i in pontuacao:
        if i in text:
            text=text.replace(i,"")
    text=text.lower()
    return text
def linha(text):
    text1=text.split()
    for k in text1:
        for j in range(len(k)-1):
            if k[j]+k[j+1]=="\n":
                k.replace(k[j],"$")
    text1=",".join(text1)
    text1=text1.replace(","," ")
    return text1
print(cleaning("\nLuís Vaz de Camões \nOs Lusíadas\nCanto Primeiro\n1\nAs armas e os barões assinalados,\nQue da ocidental praia Lusitana,\nPor mares nunca de antes navegados,\nPassaram ainda além da Taprobana,\nEm perigos e "))
print(("\nLuís Vaz de Camões \nOs Lusíadas\nCanto Primeiro\n1\nAs armas e os barões assinalados,\nQue da ocidental praia Lusitana,\nPor mares nunca de antes navegados,\nPassaram ainda além da Taprobana,\nEm perigos e "))


luís vaz de camões 
os lusíadas
canto primeiro
1
as armas e os barões assinalados
que da ocidental praia lusitana
por mares nunca de antes navegados
passaram ainda além da taprobana
em perigos e 

Luís Vaz de Camões 
Os Lusíadas
Canto Primeiro
1
As armas e os barões assinalados,
Que da ocidental praia Lusitana,
Por mares nunca de antes navegados,
Passaram ainda além da Taprobana,
Em perigos e 


In [29]:
# # test1=test.Comment
# # print(test1)
# test.Comment=test.Comment.astype("str")

# for i in test.Comment:
#     i= cleanup(str(i))
#     i=linha(i)
#     print(i)
# # print(test)
test.Comment.apply(cleanup)

0       Well we learned IUPAC it was just never tested...
1       ok what happen when i lick licky or when crazy...
2       Yeah but 300 trillion is the total global carb...
3       If only the particles outside the event horizo...
4       Heres a fun video describing more properties o...
                              ...                        
1495    So were looking at different subprocesses  You...
1496                                  Not if pure ethanol
1497    Yes it’s the same concept as ohms law but you ...
1498    It’s both It’s written by an organic chemist a...
1499    Lol No perpetual motion Just a difference of p...
Name: Comment, Length: 1500, dtype: object

In [30]:
test["comment_clean"]=test.Comment.apply(cleaning)
test["comment_clean"]=test.comment_clean.apply(linha)
# for j in test.comment_clean:
#     j=j.lower()
quimica=test.loc[test.Topic=="Chemistry",["Topic","comment_clean"]]
fisica=test.loc[test.Topic=="Physics",["Topic","comment_clean"]]
quimica
fisica
# test

,Topic,comment_clean
2,Physics,yeah but 300 trillion is the total global carb...
3,Physics,if only the particles outside the event horizo...
4,Physics,here's a fun video describing more properties ...
6,Physics,the 4 dimensional divergence of an anti-symmet...
7,Physics,how does it work i imagine the detector measur...
...,...,...
1492,Physics,i'm trying to adapt a dbscan algorithm to auto...
1493,Physics,when a ball is rolled off a horizontal cliff i...
1495,Physics,so we're looking at different sub-processes yo...
1497,Physics,yes its the same concept as ohms law but you w...


In [31]:
# preparando texto para contagem
# quimica
quimica_string=""
for comentario in quimica.comment_clean:
    quimica_string+=" "+comentario
quimica_lista=quimica_string.split()
# fisica
fisica_string=""
for comentario1 in fisica.comment_clean:
    fisica_string+=" "+comentario1
fisica_lista=fisica_string.split()
serie_quimica=pd.Series(quimica_lista)
serie_fisica=pd.Series(fisica_lista)
# contagem
tabela_quimica=serie_quimica.value_counts()
quimica_total=tabela_quimica.sum()
tabela_fisica=serie_fisica.value_counts()
fisica_total=tabela_fisica.sum()
fisica_total
tabela_fisica

the                     1454
to                       704
a                        634
of                       595
is                       588
                        ... 
memes                      1
counterintuitiveness       1
\n\nokay                   1
centered                   1
version                    1
Name: count, Length: 5055, dtype: int64

In [32]:
tabela_quimica_relativa=serie_quimica.value_counts(True)
tabela_fisica_relativa=serie_fisica.value_counts(True)


In [33]:
todas_palavra=quimica_string+fisica_string
todas_palavra=todas_palavra.split()
serie_total=pd.Series(todas_palavra)
tabela_total=serie_total.value_counts()
tabela_total_relativa=serie_total.value_counts(True)

In [34]:
def classifica_frase(string):
    prob_Frase_dadoQ=1
    prob_Frase_dadoF=1
    string=string.lower()
    string=cleaning(string)
    frase=string.split()
    probQ=tabela_quimica.sum()/tabela_total.sum()
    probF=tabela_fisica.sum()/tabela_total.sum()
    for i in frase:
        if i in tabela_quimica_relativa:
            prob_Frase_dadoQ=prob_Frase_dadoQ*((tabela_quimica[i]+1)/(tabela_quimica.sum()+len(tabela_total)))
        else:
            prob_Frase_dadoQ=prob_Frase_dadoQ*((0+1)/(tabela_quimica.sum()+len(tabela_total)))

        if i in tabela_fisica_relativa:
            prob_Frase_dadoF=prob_Frase_dadoF*((tabela_fisica[i]+1)/(tabela_fisica.sum()+len(tabela_total)))
        else:
            prob_Frase_dadoF=prob_Frase_dadoF*((0+1)/(tabela_fisica.sum()+len(tabela_total)))
    # if prob_Frase_dadoQ>0 and prob_Frase_dadoQ<1:
    #     if prob_Frase_dadoF==1:
    #         prob_Frase_dadoF=
    prob_Q_dado_frase=prob_Frase_dadoQ*probQ
    prob_F_dado_frase=prob_Frase_dadoF*probF
    lista=[prob_Q_dado_frase, prob_F_dado_frase]
    return lista


In [35]:
def texto(string):
    lista1=classifica_frase(string)
    if lista1[0]>lista1[1]:
        return "Chemistry"
    elif lista1[1]>lista1[0]:
        return "Physics"

In [36]:
def teste(li):
    resultado=[]
    true=0
    false=0
    for item in li:
        # print(item)
        testado=texto(item)
        resultado.append(testado)
        for comment in test.comment_clean:
            if comment==item:
                verdadeiro=(test[test["comment_clean"]==comment]["Topic"].values[0])
        # print(testado,verdadeiro)
        if testado==verdadeiro:

            true+=1
        elif testado!=verdadeiro:
            false+=1
    total=len(resultado)
    porcentagem=total/100
    pacerto=true/porcentagem
    return pacerto

In [37]:
lista_teste=[]
for i in fisica.comment_clean:
    lista_teste.append(i)

In [38]:
print(teste(lista_teste))

83.13069908814589


In [39]:
test.dtypes

Comment          object
Topic            object
comment_clean    object
dtype: object

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**